<a href="https://colab.research.google.com/github/danton03/Desafio-ETL-com-IA-generativa-DIO/blob/main/DesafioETL_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio DIO - Explorando IA Generativa em um Pipeline de ETL com Python

**E**xtract


In [ ]:
#Passo 1: Extrair todos os dados desejados da planilha
import pandas as pd

dataframe = pd.read_csv('SDW2023.csv')
user_ids = dataframe['UserID'].tolist() #pega todos os dados da coluna UserID
print(user_ids)

[5170, 5171, 5172]


In [ ]:
#Passo 2: Obter todos os dados de cada ID usando a API do santander dev week
import requests
import json

api_base_url = 'https://sdw-2023-prd.up.railway.app'

def get_user(id):
  response = requests.get(f'{api_base_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if((user := get_user(id)) is not None)] #compreensão de listas com walrus operator
print(json.dumps(users, indent=2))

[
  {
    "id": 5170,
    "name": "ludmilo",
    "account": {
      "id": 5501,
      "number": "18166-0",
      "agency": "0011",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 5029,
      "number": "5500 0205 6742 1816",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 9604,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Hi Ludmilo,\nInvesting is the key to building wealth. Start small and learn as you go. #investing #finance #wealth"
      }
    ]
  },
  {
    "id": 5171,
    "name": "princeso",
    "account": {
      "id": 5502,
      "number": "17132-0",
      "agency": "0011",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 5030,
      "number": "5500 0205 6742 7132",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 9605,
        "icon": "https://digitalinnovationone.github.io/s

**T**ransformação

In [ ]:
#Instalar a biblioteca do Google Palm (Bard)
!pip install google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 13.6 MB/s eta 0:00:00


In [ ]:
import google.generativeai as palm

#Criar uma variável de ambiente com a API Key da PaLM AI
palm_api_key = 'COLOQUE SUA API KEY AQUI'

#Utilizar a API da PaLM AI
palm.configure(api_key=palm_api_key)

def generate_ai_news(user):
  response = palm.generate_text(prompt=f"Imagine that you are a marketing expert. Create a message in for {user['name']}, including the name, about the importance of investing. (maximum of 100 characters)")
  return response.result.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Hi Ludmilo,
Investing is the key to building wealth. Start small and learn as you go. #investing #finance #wealth
Dear PrinceSo,

Investing is the key to building wealth and achieving your financial goals. Start early and learn from the experts.
**Pedro Ribeiro,**

Investing is the key to building wealth and securing your financial future. Start today with a small amount and watch it grow over time.


**L**oad

In [ ]:
#Atualizar a lista de news na API do Santander dev week com os dados enriquecidos

def update_user(user):
  route = str(f"{api_base_url}/users/{user['id']}")
  response = requests.put(route, json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}")

User ludmilo updated? True
User princeso updated? True
User Pedro Ribeiro updated? True
